In [1]:
# annotate poolfile after tnseq mapping with gene IDs from gff3 file
# creates a file ready for barseq counting
# fully annotated 
# removes the offending line that is super long - and causes problems in excel
# version 1.2
# sort pool by scaffold and position
# September 1, 2020

In [2]:
cd '/Users/MBP2019/annotate-piggyBAC/annotate_poolfile/'

/Users/MBP2019/annotate-piggyBAC/annotate_poolfile


In [3]:
ls

D1373_Z1.fa
D1373_Z1_fgenesh.gff3
annotate-poolFile-fgenesh/
annotation_files/
fgenesh-annotation/
new_YS2_CBS432_CDS_only_nomito_nomicron.gff3
piggyBAC_all20_poolfile_aug12_2020.txt
piggyBAC_all20_poolfile_aug12_2020.txt_fgenesh_annotated_for_BarSeq_final.txt
version1.0/
version1.1.fullyAnnotated/
version1.1.old/
version1.2.carly_fullyAnnotated/
version1.3.fgenesh_fullyAnnotated/


In [4]:
import sys
import pandas as pd
import numpy as np
import os
from Bio import SeqIO
import operator
from datetime import datetime
import collections
import matplotlib.pyplot as plt
import argparse

In [5]:
# Load poolfile 

poolFileName = 'piggyBAC_all20_poolfile_aug12_2020.txt'

try:
    with open(poolFileName, 'rb') as poolFileHandle:
        poolFrame = pd.read_csv(poolFileHandle,sep='\t')
        poolFileHandle.close()
        poolFrame.dropna(how='all')
        statusUpdate =  "Read "+ str(len(poolFrame)) + " barcodes from "+poolFileName
        print(statusUpdate)
except IOError:
    statusUpdate =  "Could not read file: "+poolFileName
    print(statusUpdate)
    sys.exit()

Read 548129 barcodes from piggyBAC_all20_poolfile_aug12_2020.txt


In [6]:
# convert scaffolds to dictionary of SeqRecord objects

genomeFiles = 'D1373_Z1.fa'
scaffoldSequences = SeqIO.to_dict(SeqIO.parse(genomeFiles, "fasta"))


print("Sorting pool entries by location.")
poolFrame.sort_values(['scaffold','pos'],ascending=[1,1],inplace=True)
        
# sort scaffolds by length

scaffoldLengths = {}
totalGenome = 0
for scaffold in scaffoldSequences:
        scaffoldLengths[scaffold] = len(scaffoldSequences[scaffold].seq)
        totalGenome+=len(scaffoldSequences[scaffold].seq)

sorted_scaffoldLengths = sorted(scaffoldLengths.items(), key=operator.itemgetter(1), reverse=True)
scaffoldOrder = []
for scaff in sorted_scaffoldLengths:
    scaffoldOrder.append(scaff[0])
    
print(f'Sorted scaffolds by length: {scaffoldOrder}')

Sorting pool entries by location.
Sorted scaffolds by length: ['scchrIV', 'sp4', 'scchrXV', 'scchrVII', 'scchrXII', 'sp7', 'sp15', 'sp12', 'sp16', 'scchrXVI', 'scchrXIII', 'sp13', 'scchrII', 'sp2', 'scchrXIV', 'sp14', 'scchrX', 'sp10', 'sp11', 'scchrXI', 'sp5', 'scchrV', 'scchrVIII', 'sp8', 'scchrIX', 'sp9', 'scchrIII', 'sp3', 'sp6', 'scchrVI', 'scchrI', 'sp1', 'scchrMito']


In [7]:
# Load gff file

gffFileName = 'new_YS2_CBS432_CDS_only_nomito_nomicron.gff3'

try:
    with open(gffFileName, 'rb') as FileHandle: # why rb mode?
        genestext = FileHandle.readlines()
        FileHandle.close()
        statusUpdate = "Read "+str(len(genestext))+" features from "+gffFileName
        print(statusUpdate)
except IOError:
    statusUpdate = "Could not read file: "+gffFileName
    print(statusUpdate)
    sys.exit()

Read 51372 features from new_YS2_CBS432_CDS_only_nomito_nomicron.gff3


In [8]:
# Parse GFF file for gene locations and attributes

statusUpdate =  "Parsing GFF for gene locations and attributes"
print(statusUpdate)



# Build list of RNAs
# parse GFF into OrderedDict called Genes

Genes = collections.OrderedDict()
Counter = 0
for genesline in genestext:
    genesfields = genesline.decode().split('\t')
    if len(genesfields) == 9:
        scaffold,evidence,featureType,start,end,flag,strand,frame,comments = genesfields
        #translate start/end to zero based indexing with non-inclusive end
        start = int(start)-1
        end = int(end)
        if featureType == 'mRNA':
            notes = dict((k.strip(), v.strip()) for k,v in (item.split('=') for item in comments.split(';')))
                    
            Genes[notes['ID']] = {'scaffold':scaffold,
                                        'start':int(start),
                                        'end':int(end),
                                        'strand':strand,
                                        'exon':[],
                                        'intron':[],
                                        'Parent':notes['Parent'],
                                        }
        elif featureType == 'CDS':
            notes = dict((k.strip(), v.strip()) for k,v in (item.split('=') for item in comments.split(';')))
            if len(Genes[notes['Parent']]['exon']) > 0:
                Genes[notes['Parent']]['intron'].append([Genes[notes['Parent']]['exon'][-1][1]+1,start-1])
            Genes[notes['Parent']]['exon'].append([start,end])

Parsing GFF for gene locations and attributes


In [9]:
# example, has two exons and one intron
Genes['nbisL2-cds-4605']

{'scaffold': 'scchrXIII',
 'start': 460,
 'end': 4684,
 'strand': '-',
 'exon': [[460, 3791], [3890, 4684]],
 'intron': [[3792, 3889]],
 'Parent': 'nbis-gene-4784'}

In [10]:
# Build list of locations in the genome and in genic and non-genic regions
# should have intron / exon / intergenic ?   what happens to them?
# and for fitness calculation - what happens to exon/intron/intergenic?
# does it sort by nearest gene?



locationTypes = {}
nearestGeneDict = {}
for scaffold in scaffoldOrder:
    locationTypes[scaffold] = ['intergenic']*scaffoldLengths[scaffold]
    nearestGeneDict[scaffold] = ['intergenic']*scaffoldLengths[scaffold]

    
for locationType in ['intron','exon']:
    for Gene in Genes:
        for entry in Genes[Gene][locationType]:
            for position in range(entry[0],entry[1]):
                locationTypes[Genes[Gene]['scaffold']][position] = locationType
                nearestGeneDict[Genes[Gene]['scaffold']][position] = Gene


# Count up locations in promoters, exons, etc
locationsByType = {}
locationTypeTotals = {}
insertionTypeTotals = {}
locationTypeClasses = ['intron', 'exon','intergenic']
for locationType in locationTypeClasses:
    locationTypeTotals[locationType] = 0
    insertionTypeTotals[locationType] = 0
    locationsByType[locationType] = {}
    for scaffold in scaffoldOrder:
        locationsByType[locationType][scaffold] = []
        for location in range(0,scaffoldLengths[scaffold]):
            if locationTypes[scaffold][location] == locationType:
                locationsByType[locationType][scaffold].append(location)
                locationTypeTotals[locationType]+=1

In [11]:
# Annotate poolfile

# Count up insertions in promoters, exons, etc.
codingFractions = []
nearestGeneIDs = []
insertionTypesOrdered = []
statusUpdate =  "Classifying insertion locations"
print(statusUpdate)

for index in poolFrame.index:
    scaffold = poolFrame.at[index,'scaffold']
    position = int(poolFrame.at[index,'pos'])-1
    insertionType = locationTypes[scaffold][position]
    insertionTypeTotals[insertionType] += 1
    insertionTypesOrdered.append(insertionType)

    nearestGene = nearestGeneDict[scaffold][position]
    
    if nearestGene == 'intergenic':
        nearestGeneIDs.append('intergenic')
    else:
        nearestGeneIDs.append(Genes[nearestGene]['Parent'])
        
    if insertionType in ['intron','exon']:
        flattened_exons = [item for sublist in Genes[nearestGene]['exon'] for item in sublist]
        codingStart = min(flattened_exons)
        codingStop = max(flattened_exons)
        codingFractions.append(int((float(position - codingStart) / (codingStop - codingStart)) * 100))

    else:
        codingFractions.append("NA")    


poolFrame['InsertionType'] = insertionTypesOrdered
poolFrame['NearestGene'] = nearestGeneIDs
poolFrame['CodingFraction'] = codingFractions



Classifying insertion locations


In [12]:
# compute LocalGCpercent and add column


# Check GC content around insertion sites

print('compute GC content around insertion sites')
scaffoldArr = poolFrame['scaffold'].values
posArr = poolFrame['pos'].values
flankingWindow = 50
GCpcts = []
scaffGC = []
for idx, scaffold in enumerate(scaffoldArr):
    pos = int(posArr[idx])
    limits = [max(0,pos-flankingWindow),min(pos+flankingWindow,scaffoldLengths[scaffold])]
    localSeq = str(scaffoldSequences[scaffold].seq[limits[0]:limits[1]])
    Ccount = localSeq.count('C')
    Gcount = localSeq.count('G')
    GCpcts.append(100*(Gcount + Ccount)/len(localSeq))
poolFrame['LocalGCpercent'] = GCpcts



    

compute GC content around insertion sites


In [13]:
poolFrame['barcode'].isna().value_counts()

False    548129
Name: barcode, dtype: int64

In [14]:
# add rest of columns needed ID	AlternateID	Annotation
gene_to_name_carly = pd.read_csv('annotation_files/gene_to_name_carly.txt',header=0,sep='\t',low_memory=False)
SGD_annotation = pd.read_csv('annotation_files/SGD_annotation_sc_sp.txt',header=0,sep='\t',low_memory=False)

merge1 = pd.merge(poolFrame, gene_to_name_carly, left_on = 'NearestGene', right_on = 'geneID', how='left')
merge2 = pd.merge(merge1, SGD_annotation, left_on = 'name', right_on = 'sc_sp_name', how='left')

merge2.rename(columns={'geneID': 'ID', 'name': 'AlternateID', 'annotation': 'Annotation'},inplace=True)

merge2.drop(columns=['name_1', 'sc_sp_name'],inplace=True)

# fix the offending line
# bigline = merge2[merge2['barcode'] == 'TGTAATTTTCGCATCAACCA']['All genomic mappings']

merge2.at[432601,'All genomic mappings'] = 'none'

merge2.to_csv(poolFileName+"_carly_annotated_for_BarSeq_final.txt",sep="\t",index=False)


statusUpdate =  "Wrote poolfile with gene information to "+ poolFileName+'_carly_annotated_for_BarSeq_final.txt'
print(statusUpdate)

Wrote poolfile with gene information to piggyBAC_all20_poolfile_aug12_2020.txt_carly_annotated_for_BarSeq_final.txt
